## From interactive programming to production ready code

### Imports

In [1]:
from luigi.parameter import IntParameter, Parameter
from luigi import LocalTarget, Task

## Task No.1: Check for existing dataset

The provided Docker Image already contains the dataset. This tasks just checks if everything that is needed is at the right place.

*Output*: A folder containing the images

In [3]:
class DatasetExists(Task):
    dataset = Parameter(default='../../fruits')
    
    def output(self):
        return LocalTarget(self.dataset)

## Task No.2: Create a preprocessing configuration

The configuration for the deep-learning model is essentially the Keras ImageDataGenerator. For the sake of simplicity we do not parameterize this task. But we can grasp the idea how to do it.

*Input*: Nothing required <br>
*Output*: A pickled ImageDataGenerator

In [4]:
from keras import *

# image processing
from keras.preprocessing import image as image_utils
from keras.preprocessing.image import img_to_array

# build your own nets
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

Using TensorFlow backend.


In [9]:
import pickle

In [12]:
class Configure(Task):
    pickle_path = Parameter(default="/keras2production/notebooks/2-luigi/exercise-dataset/pickle_files")
    
    def output(self):
        train_data_gen = ImageDataGenerator(rescale = 1/255)
        pickle.dump(train_data_gen, open(self.pickle_path+"/im_generator.p", "wb" ) )
        


## Task No.3: Run the baseline validation

This task runs the baseline validation and saves it to a file. The same as before, flexibility can be greatly enhanced by als versioning the baseline validation.

*Input*: ExtractDataset, Configure <br>
*Output*: A JSON-File containing the baseline accuracy

In [6]:
class BaselineValidation(Task):
    

## Task No.4: Train the deep learning model

Task No.4 trains a Keras model and persists it to the filesystem.

*Input*: ExtractDataset, Configure <br>
*Output*: A .h5 file representing the model architecture and its weights

In [7]:
def image_array_gen(im_gen, train_image_files_path, img_width, img_height, batch_size, fruit_list):
    return im_gen.flow_from_directory(
                train_image_files_path,
                target_size = (img_width, img_height),
                class_mode = 'categorical',
                classes = fruit_list,
                color_mode = 'rgb', 
                batch_size = batch_size,
                seed = 42)

def create_model(): 
    model = Sequential()

    # first hidden layer
    model.add(Conv2D(32, (3, 3), padding = "same", input_shape = (img_width, img_height, channels)))
    model.add(Activation('relu'))

    # second hidden layer
    model.add(Conv2D(16, (3, 3), padding = "same"))
    model.add(LeakyReLU(alpha = 0.5))
    model.add(BatchNormalization())

    # max pooling
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Dropout(0.25))

    # Flatten max filtered output into feature vector 
    # and feed into dense layer
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    # Outputs from dense layer are projected onto output layer
    model.add(Dense(output_n))
    model.add(Activation('softmax'))

    return model

class TrainModel(Task):
    train_image_files_path = '../../fruits/Training/'
    valid_image_files_path = '../../fruits/Test/'
    pickle_path = Parameter(default="/keras2production/notebooks/2-luigi/exercise-dataset/pickle_files")
    img_width = Parameter(default=20)
    img_height = Parameter(default=20)
    batch_size = Parameter(default=32)
    fruit_list = Parameter(default=["Apricot", "Avocado", "Banana", "Clementine", "Cocos", "Kiwi", "Lemon", "Limes", 
                                    "Mandarine", "Orange", "Peach", "Pineapple", "Plum", "Pomegranate", "Raspberry", "Strawberry"])
    
    def requires(self):
        yield DatasetExists()
        yield Configure()
        
    def output(self):
        return LocalTarget(self.model_path+'/model.h5')
    
    def run(self):
        im_generator = pickle.load(open(self.pickle_path+'/im_generator.p','rb'))
        train_image_array_gen = image_array_gen(im_generator, self.train_image_files_path, 
                                                self.img_width, self.img_height, self.batch_size,self.fruit_list)
        valid_image_array_gen = image_array_gen(im_generator, self.valid_image_files_path,
                                                self.img_width, self.img_height, self.batch_size,self.fruit_list)
        train_samples = train_image_array_gen.n
        valid_samples = valid_image_array_gen.n
        model = create_model()
        model.compile(loss = 'categorical_crossentropy', 
              optimizer = RMSprop(lr = 0.0001, decay = 1e-6),
              metrics = ['accuracy'])
        model.fit_generator(
            train_image_array_gen,
            steps_per_epoch = int(train_samples / self.batch_size), 
            epochs = epochs, 
            validation_data = valid_image_array_gen,
            validation_steps = int(valid_samples / self.batch_size),
            verbose = 1) 
        model.save(self.output().path)
        


## Task No.5: Evaluate the model

The last task evaluates our model and - if it surpasses the baseline accuracy - saves the evaluation results to the filesystem. Let the task crash if the model does not perform well enough. It's worth an exception!

*Input*: ExtractDataset, Configure, TrainModel, BaselineValidation<br>
*Output*: A JSON file containing the evaluation results


In [16]:
class Evaluate(Task):
    pass

## Surprise Task No.6: Deploy to TensorFlow-Serving

The Keras model is performing well. Let's deploy it to TensorFlow Serving.

It can be loaded with TensorFlow Serving by the following command:
tensorflow_model_server --model_name="keras_model" --model_base_path="serving/keras_model"

*Input*: TrainModel, Evaluate </br>
*Output*: The TensorFlow-Graph and its weights

In [9]:
class Export(Task):
    pass